In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [78]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from sklearn.metrics import precision_score, f1_score, confusion_matrix, recall_score, accuracy_score


In [57]:
data = pd.read_csv('file_mapping.csv')
data = data.sample(frac=1)
data.head()

,Filename,Brand Name,Label,Tagline
496,genLogoOutput\Mtn dew\scal_000003.jpg,Mtn dew,Fake,position vitamin_A fiddling hayseed inwards yo...
295,genLogoOutput\Oral b\000004.jpg,Oral b,Fake,The magnate toothbrush Sir_Thomas_More tooth_d...
335,genLogoOutput\Instagram\scal_000004.jpg,Instagram,Fake,trance and partake the earthly_concern 's pres...
107,output\Hp\000004.jpg,Hp,Genuine,The Great Tasting Gel!
638,genLogoOutput\Goya oil\scal_000001.jpg,Goya oil,Fake,If IT 's Francisco_Jose_de_Goya_y_Lucientes .....


In [58]:
data.shape

(825, 4)

In [59]:
data.isnull().sum()

Filename       0
Brand Name     0
Label          0
Tagline       12
dtype: int64

In [60]:
data.Label.value_counts()

Fake       550
Genuine    275
Name: Label, dtype: int64

In [62]:
data['Label'].replace(('Fake','Genuine'),(0,1), inplace = True)
data.head()

,Filename,Brand Name,Label,Tagline
496,genLogoOutput\Mtn dew\scal_000003.jpg,Mtn dew,0,position vitamin_A fiddling hayseed inwards yo...
295,genLogoOutput\Oral b\000004.jpg,Oral b,0,The magnate toothbrush Sir_Thomas_More tooth_d...
335,genLogoOutput\Instagram\scal_000004.jpg,Instagram,0,trance and partake the earthly_concern 's pres...
107,output\Hp\000004.jpg,Hp,1,The Great Tasting Gel!
638,genLogoOutput\Goya oil\scal_000001.jpg,Goya oil,0,If IT 's Francisco_Jose_de_Goya_y_Lucientes .....


In [63]:
X = []

for i in range(data.shape[0]):
  filepath = data.iloc[i]['Filename']  
  filepath = filepath.replace(('\\'),('//'))
  img_path = f'drive//MyDrive//{filepath}'

  image = cv2.imread(img_path) 
  image = cv2.resize(image, (70,70), interpolation= cv2.INTER_LINEAR)  
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)        
  # Append the image and its corresponding label to the output
  X.append(image)

X = np.array(X, dtype = 'float32') / 255.0

In [64]:
Y = data['Label']
Y = pd.get_dummies(Y)


(825, 2)

In [65]:
Y

,0,1
496,1,0
295,1,0
335,1,0
107,0,1
638,1,0
...,...,...
347,1,0
319,1,0
137,0,1
212,0,1


In [66]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [67]:
optimizer = tf.keras.optimizers.Adam()
early_stopping = [tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', patience = 10)]


model = Sequential()
model.add(Conv2D(64, kernel_size = (2,2), strides=2, padding="same", activation = 'relu', input_shape = (70,70,3)))
model.add(Conv2D(64, kernel_size = (2,2), strides=2, padding="same", activation  = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), padding="same"))
model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size = (2,2), strides=2, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), padding="same"))
model.add(Conv2D(32, kernel_size = (2,2), strides=2, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), padding="same"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = "categorical_crossentropy", metrics = "accuracy", optimizer = optimizer)

r = model.fit(xtrain, ytrain, epochs = 300, batch_size=32, callbacks = early_stopping)

Epoch 1/300
21/21 [==============================] - 41s 38ms/step - loss: 0.6525 - accuracy: 0.6455
Epoch 2/300
21/21 [==============================] - 1s 40ms/step - loss: 0.6248 - accuracy: 0.6636
Epoch 3/300
21/21 [==============================] - 1s 38ms/step - loss: 0.5427 - accuracy: 0.6545
Epoch 4/300
21/21 [==============================] - 1s 38ms/step - loss: 0.5126 - accuracy: 0.6758
Epoch 5/300
21/21 [==============================] - 1s 39ms/step - loss: 0.5003 - accuracy: 0.6500
Epoch 6/300
21/21 [==============================] - 1s 39ms/step - loss: 0.4922 - accuracy: 0.6667
Epoch 7/300
21/21 [==============================] - 1s 39ms/step - loss: 0.4893 - accuracy: 0.6742
Epoch 8/300
21/21 [==============================] - 1s 55ms/step - loss: 0.4838 - accuracy: 0.6652
Epoch 9/300
21/21 [==============================] - 1s 63ms/step - loss: 0.4794 - accuracy: 0.7242
Epoch 10/300
21/21 [==============================] - 1s 52ms/step - loss: 0.4592 - accuracy: 0.736

In [68]:
ypred = model.predict(xtest)
ypred = np.argmax(ypred, axis = 1)
ytest_max = np.argmax(ytest.to_numpy(), axis = 1)

6/6 [==============================] - 0s 19ms/step


In [77]:
print(f'Accuracy Score: {accuracy_score(ytest_max, ypred)}')
print(f'Precision Score: {precision_score(ytest_max, ypred)}')
print(f'Recall Score: {recall_score(ytest_max, ypred)}')
print(f'F1 Score: {f1_score(ytest_max, ypred)}')

Accuracy Score: 0.9151515151515152
Precision Score: 0.8823529411764706
Recall Score: 0.8490566037735849
F1 Score: 0.8653846153846154


In [79]:
optimizer = tf.keras.optimizers.Adam()
early_stopping = [tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', patience = 10)]


model = Sequential()
model.add(Conv2D(128, kernel_size = (2,2), strides=2, padding="same", activation = 'relu', input_shape = (70,70,3)))
model.add(Conv2D(128, kernel_size = (2,2), strides=2, padding="same", activation  = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), padding="same"))
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size = (2,2), strides=2, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), padding="same"))
model.add(Conv2D(64, kernel_size = (2,2), strides=2, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2), padding="same"))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = "categorical_crossentropy", metrics = "accuracy", optimizer = optimizer)

r = model.fit(xtrain, ytrain, epochs = 300, batch_size=32, callbacks = early_stopping)

Epoch 1/300
21/21 [==============================] - 5s 155ms/step - loss: 0.6428 - accuracy: 0.6561
Epoch 2/300
21/21 [==============================] - 2s 97ms/step - loss: 0.5884 - accuracy: 0.6576
Epoch 3/300
21/21 [==============================] - 2s 96ms/step - loss: 0.5120 - accuracy: 0.6545
Epoch 4/300
21/21 [==============================] - 2s 98ms/step - loss: 0.4979 - accuracy: 0.6485
Epoch 5/300
21/21 [==============================] - 2s 106ms/step - loss: 0.4981 - accuracy: 0.6591
Epoch 6/300
21/21 [==============================] - 4s 170ms/step - loss: 0.4881 - accuracy: 0.6727
Epoch 7/300
21/21 [==============================] - 2s 96ms/step - loss: 0.4950 - accuracy: 0.6742
Epoch 8/300
21/21 [==============================] - 2s 112ms/step - loss: 0.4851 - accuracy: 0.6591
Epoch 9/300
21/21 [==============================] - 2s 95ms/step - loss: 0.4824 - accuracy: 0.6545
Epoch 10/300
21/21 [==============================] - 2s 97ms/step - loss: 0.4784 - accuracy: 0.

In [80]:
ypred = model.predict(xtest)
ypred = np.argmax(ypred, axis = 1)
ytest_max = np.argmax(ytest.to_numpy(), axis = 1)

6/6 [==============================] - 0s 22ms/step


In [81]:
print(f'Accuracy Score: {accuracy_score(ytest_max, ypred)}')
print(f'Precision Score: {precision_score(ytest_max, ypred)}')
print(f'Recall Score: {recall_score(ytest_max, ypred)}')
print(f'F1 Score: {f1_score(ytest_max, ypred)}')

Accuracy Score: 0.8848484848484849
Precision Score: 0.7833333333333333
Recall Score: 0.8867924528301887
F1 Score: 0.8318584070796461
